# 한국어 형태소 분석

## 학습 목표

- 한국어 형태소 분석
- Word Piece Model
- CountVectorizer에서 tokenizer 수동 설정


## 다음 뉴스

### 기사 링크 모으기

In [1]:
import requests

In [2]:
import lxml.html

In [3]:
import tqdm

In [4]:
search_url = 'https://search.daum.net/search?nil_suggest=btn&w=news&cluster=y&q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&cp=16yGc-mR1Rz5JT4-UZ&cpname=%EC%A0%84%EC%9E%90%EC%8B%A0%EB%AC%B8&DA=PGD&p={}'

In [5]:
urls = []
for page in tqdm.tnrange(1, 30):
    res = requests.get(search_url.format(page))
    root = lxml.html.fromstring(res.text)
    for link in root.cssselect('.date a'):
        urls.append(link.attrib['href'])

HBox(children=(IntProgress(value=0, max=29), HTML(value='')))

### 기사 본문 모으기

In [6]:
articles = []
for url in tqdm.tqdm_notebook(urls):
    try:
        res = requests.get(url, timeout=1)
    except requests.Timeout:
        continue
    root = lxml.html.fromstring(res.text)
    content = root.cssselect('.article_view')[0]
    article = content.text_content().strip()
    articles.append({'url': url, 'article': article})

HBox(children=(IntProgress(value=0, max=299), HTML(value='')))

## 파일로 저장

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(articles)

In [9]:
df.head()

,article,url
0,전자부품연구원(KETI)은 '산업인공지능 전문인력 양성사업' 주관기관으로 선정됐다고...,http://v.media.daum.net/v/20190425110407280
1,코난테크놀로지는 지난 17일 서울 엘타워에서 사외이사와 임직원 등 관계자 200여 ...,http://v.media.daum.net/v/20190425090103284
2,서울 영등포구 LG 트윈타워\n \n \n ...,http://v.media.daum.net/v/20190422143106509
3,영국 정부가 글로벌 기업 인공지능 스피커에 1만2000여건 정보를 개방했다. 컴퓨터...,http://v.media.daum.net/v/20190422135204089
4,전자신문사와 애자일소다가 공동 주최하는 '인공지능 엔터프라이즈 2019' 콘퍼런스가...,http://v.media.daum.net/v/20190425165410114


In [13]:
df.to_csv('인공지능.csv')

### 파일에서 불러오기

In [14]:
df = pd.read_csv('인공지능.csv', engine='python', index_col=0, encoding='utf8')

### 본문 보기

In [10]:
df.loc[0, 'article']

"전자부품연구원(KETI)은 '산업인공지능 전문인력 양성사업' 주관기관으로 선정됐다고 25일 밝혔다.\n         산업인공지능은 제조업과 제조 관련 전후방 서비스 산업에 인공지능(AI)을 적용, 제조업 고도화로 부가가치를 창출하는 기술을 일컫는다.\n         KETI는 포항공대, 고려대와 함께 산업인공지능에 특화된 석·박사 전문인력을 양성하는 프로그램을 운영할 계획이다.\n         제조현장 문제해결형 교육을 위한 산업데이터 교육 플랫폼 구축, 스마트제조 해커톤 대회 개최, 산업인공지능 교과목 개발 등으로 전문인력을 양성할 방침이다.\n         사업 총괄책임자인 송병훈 KETI 센터장은 “산업인공지능 인력양성 프로그램은 국내외 기업이 보유한 산업 데이터를 대학과 공유해 제조현장 난제를 해결하는 국내 최초 스마트공장 전문인력 양성코스”라며 “향후 스마트제조혁신 분야를 이끌 인력 양성으로 국내 제조업 경쟁력을 한 단계 끌어올리는데 앞장설 것”이라 밝혔다.\n         \n          \n          \n           KETI가 운영 중인 경기도 스마트제조혁신센터 내 데모공장(자료: KETI)\n          \n         \n         윤건일 전자/부품 전문기자 benyun@etnews.com"

## KoNLPy 설치

### 윈도에서 설치 준비

1. Java 설치
2. 환경변수 설정
3. JPype1 설치

#### 자바 설치

- 윈도의 경우 시작 버튼을 우클릭 후 '시스템'을 선택
- '시스템 종류' 항목에 '64비트'인지 '32비트'인지 확인
- http://java.com 접속
- 운영체제가 64비트면 64비트 Java다운로드 (32비트면 32비트 Java)
- 설치

#### 환경변수 설정

1. 윈도 시작버튼
2. 고급 시스템 설정 검색후 선택
3. 고급 선택
4. 환경변수 선택
5. 시스템 변수나 사용자 변수에서 `JAVA_HOME`을 찾는다. 없으면 새로 만든다.
6. `JAVA_HOME`에는 Java가 설치된 위치(예:`C:\Program Files\Java\1.8.0_131`)를 입력

Java가 설치된 버전의 마지막 숫자는 버전에 따라 다르므로 확인 필요

#### JPype1 설치

1. Unofficial Windows Binaries for Python Extension Packages
  - https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype
2. 사양에 맞는 파일을 받는다
  - `cp36`: 파이썬 3.6
  - `cp37`: 파이썬 3.7
  - `win32`: 32비트 윈도
  - `win_amd64`: 64 비트 윈도
3. 다운로드 폴더에서 cmd 창 열기
4. `pip install JPype1‑0.6.2‑cp37‑cp37m‑win_amd64.whl`과 같이 다운로드

### konlpy 설치

In [16]:
!pip install konlpy

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## KoNLPy를 이용한 한국어 형태소 분석

In [11]:
from konlpy.tag import Okt

In [12]:
tagger = Okt()

In [13]:
# pos - 품사 분석
tagger.pos('아버지가방에들어가신다.')

[('아버지', 'Noun'),
 ('가방', 'Noun'),
 ('에', 'Josa'),
 ('들어가신다', 'Verb'),
 ('.', 'Punctuation')]

In [14]:
tagger.nouns(df.loc[0, 'article'])

['전자부품',
 '연구원',
 '은',
 '산업',
 '인공',
 '지능',
 '전문',
 '인력',
 '양성',
 '사업',
 '주관',
 '기관',
 '선정',
 '산업',
 '지능',
 '제조업',
 '제조',
 '관련',
 '후방',
 '서비스',
 '산업',
 '인공',
 '지능',
 '적용',
 '제조업',
 '고도화',
 '부가가치',
 '창',
 '출하',
 '기술',
 '포항',
 '공대',
 '고려대',
 '산업',
 '지능',
 '특화된',
 '석',
 '박사',
 '인력',
 '양성',
 '프로그램',
 '운영',
 '계획',
 '제조',
 '현장',
 '문제해결',
 '교육',
 '위',
 '산업',
 '데이터',
 '교육',
 '플랫폼',
 '구축',
 '스마트',
 '제조',
 '해커',
 '톤',
 '대회',
 '개최',
 '산업',
 '인공',
 '지능',
 '교과목',
 '개발',
 '등',
 '인력',
 '양성',
 '방침',
 '사업',
 '총괄',
 '책임자',
 '송병훈',
 '센터',
 '산업',
 '인공',
 '지능',
 '인력',
 '양성',
 '프로그램',
 '국내외',
 '기업',
 '보유',
 '산업',
 '데이터',
 '대학',
 '공유',
 '제조',
 '현장',
 '난제',
 '해결',
 '국내',
 '최초',
 '스마트',
 '공장',
 '전문',
 '인력',
 '양성',
 '코스',
 '라며',
 '향후',
 '스마트',
 '제조',
 '혁신',
 '분야',
 '인력',
 '양성',
 '국내',
 '제조업',
 '경쟁력',
 '단계',
 '것',
 '운영',
 '중인',
 '경기도',
 '스마트',
 '제조',
 '혁신',
 '센터',
 '내',
 '데모',
 '공장',
 '자료',
 '윤건',
 '일',
 '전자',
 '부품',
 '전문',
 '기자']

### konlpy의 형태소 분석기 종류

In [15]:
from konlpy.tag import Kkma, Komoran, Hannanum, Okt

### 한나눔

KAIST 최기선 교수 연구팀 (1999)

In [16]:
from konlpy.tag import Hannanum

In [17]:
tag_hnn = Hannanum()

In [18]:
tag_hnn.pos('아버지가방에 들어가신다.')

[('아버지가방', 'N'),
 ('에', 'J'),
 ('들', 'P'),
 ('어', 'E'),
 ('가', 'P'),
 ('시ㄴ다', 'E'),
 ('.', 'S')]

### 꼬꼬마

서울대 IDS 연구실 (2009)

In [ ]:
from konlpy.tag import Kkma

In [44]:
tag_kkm = Kkma()

In [45]:
tag_kkm.pos('아버지가방에들어가신다.')

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

### 코모란

샤인웨어 (2013)

In [168]:
from konlpy.tag import Komoran

In [169]:
tag_kmr = Komoran()

In [43]:
tag_kmr.pos('아버지가방에들어가신다.')

[('아버지', 'NNG'),
 ('가방', 'NNP'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('시', 'EP'),
 ('ㄴ다', 'EF'),
 ('.', 'SF')]

## Okt

트위터 코리아에서 공개

In [39]:
from konlpy.tag import Okt

In [40]:
tag_okt = Okt()

In [41]:
tag_okt.pos('아버지가방에들어가신다.')

[('아버지', 'Noun'),
 ('가방', 'Noun'),
 ('에', 'Josa'),
 ('들어가신다', 'Verb'),
 ('.', 'Punctuation')]

## 형태소 분석의 문제

- 신조어
- 합성어
- 오탈자, 맞춤법

### 신조어

In [60]:
tag_kmr.pos('최순실의 신발이 벗겨졌다.')

[('최순', 'NNP'),
 ('실', 'NNP'),
 ('의', 'JKG'),
 ('신발', 'NNG'),
 ('이', 'JKS'),
 ('벗겨지', 'VV'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF')]

### 합성어

In [173]:
tag_kmr.pos('4차 산업혁명')

[('4', 'SN'), ('차', 'NNB'), ('산업', 'NNG'), ('혁명', 'NNP')]

### 오탈자, 맞춤법

In [59]:
tag_kmr.pos('몇일후에 보자.')

[('몇', 'MM'),
 ('일', 'NNP'),
 ('후에', 'NNP'),
 ('보', 'VX'),
 ('자', 'EF'),
 ('.', 'SF')]

## Word Piece Model

### 설치

In [ ]:
!pip install subword-nmt

### 학습

In [17]:
import io

In [15]:
from subword_nmt.learn_bpe import learn_bpe

In [18]:
with open('인공지능BPE.txt', 'w', encoding='utf8') as outfile:
    infile = io.StringIO(' '.join(df['article']))
    learn_bpe(infile, outfile, 1000)

### 학습 결과

In [19]:
with open('인공지능BPE.txt', encoding='utf8') as f:
    for line in f.readlines()[:30]:
        print(line, end='')

#version: 0.2
다 .</w>
으 로</w>
했 다.</w>
하 는</w>
에 서</w>
기 술
한 다.</w>
인 공
인공 지
하 고</w>
데 이
이 다.</w>
있 다.</w>
서 비
A I
A I</w>
2 0
인공지 능
기 업
서비 스
다 ”
개 발
데이 터
연 구
정 보
인공지 능</w>
산 업
이 라
있 는</w>


### 적용

In [20]:
from subword_nmt.apply_bpe import BPE

In [21]:
with open('인공지능BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

In [22]:
bpe.process_line(df.loc[0, 'article'])

'한~ 솔~ 인~ 티~ 큐~ 브~ 의 커~ 뮤~ 니~ 케이~ 션 브~ 랜~ 드 ‘~ 아이~ 작~ ’~ 은 옴~ 니~ 채~ 널~ 과 인공지능(AI) 기술 도입~ 이 어려~ 운 컨~ 택~ 센터~ 를 위한 클라우드 서비스를 선보~ 인~ 다~ .~ \n 기업~ 들은 빠르게 변화~ 하는 시장 환경 속~ 에서 고객~ 의 니~ 즈~ 를 수~ 용~ 할 수 있는 유~ 연~ 한 플랫폼을 필요~ 로 한다. 과~ 거~ 에는 전~ 화~ 라는 하나~ 의 채~ 널~ 로 모~ 이~ 던 고객~ 의 목~ 소~ 리~ 가 최~ 근~ 에는 웹~ , 모바일 플랫폼 발전~ 에 따라 다양~ 화~ 되고 있다. 다양한 채~ 널~ 은 시대~ 에 따라 새~ 로 추가~ 되~ 기~ 도 하고 사~ 라~ 지~ 기~ 도 한다. 카카오~ 톡~ 이 문~ 자 메~ 시~ 지~ (~ SM~ S~ ) 사용~ 량을 넘~ 어~ 선 것이 대표~ 적인 예~ 이~ 다~ .~ \n 이~ 렇~ 듯 환경~ 과 고객~ 의 변화~ 는 시간~ 이 갈~ 수~ 록 가~ 속~ 화~ 되고 예측~ 이 불~ 가~ 하다. 앞으로 기업~ 들은 어~ 떻~ 게 이러한 환경~ 에 대응~ 하여 최적~ 의 고객 경험을 제공~ 할 수 있~ 을~ 까~ ?~ \n 과~ 거~ 와 같~ 이 대~ 규모 투자를 통한 인프~ 라 구축 방식~ 으로는 시장 변~ 화 속~ 도~ 에 따~ 라~ 갈 수 없~ 다. 특히 신~ 기술 도입~ 을 통한 서비스~ 는 론~ 칭 이후 시장~ 의 검~ 증 과정~ 이 필~ 수~ 적~ 이다. 고객~ 의 소~ 리~ 에 따라 서비스를 지속~ 적으로 개선~ 해~ 나가~ 는 과정~ 이 필요~ 하다. AI~ CC~ (~ 인공지능 컨~ 택~ 센터~ )~ 의 경~ 우~ 도 마~ 찬~ 가~ 지~ 이~ 다~ .~ \n 인공지능 기술이 최근 각~ 광~ 받고 있~ 음~ 에도 불~ 구~ 하고 기업~ 에서는 선~ 뜻 AI~ CC~ 로~ 의 전환~ 을 결정~ 하기~ 가 어~ 렵~ 다. 기술~ 의 장~ 벽~ , 높은 초~ 기 비~ 용~ 은 차~ 치~ 하~ 더~ 라~ 도 그 효과~ 성~ 에

## konlpy를 이용한 한국어 TDM

In [23]:
from konlpy.tag import Okt

In [24]:
tag_okt = Okt()

In [25]:
def tokenizer_okt(text):
    pos = tag_okt.pos(text)
    return [w for w, t in pos if len(w) > 1]

### TDM 만들기

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
cv_okt = CountVectorizer(max_features=1000, tokenizer=tokenizer_okt)

In [28]:
tdm_okt = cv_okt.fit_transform(df['article'])

### 빈도 표

In [29]:
freq_okt = pd.DataFrame({
    'word': cv_okt.get_feature_names(),
    'n': tdm_okt.sum(axis=0).flat
})

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


### 결과 보기

In [30]:
freq_okt.sort_values('n').tail(10)

,word,n
690,있다,749
668,인공,792
923,하는,854
796,지능,858
559,에서,970
191,기술,1011
959,했다,1107
31,ai,1114
629,으로,1274
0,\n,2228


## WPM을 이용한 한국어 TDM

In [19]:
def tokenizer_wpm(text):
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

### TDM 만들기

In [32]:
cv_wpm = CountVectorizer(max_features=1000, tokenizer=tokenizer_wpm)

In [33]:
tdm_wpm = cv_wpm.fit_transform(df['article'])

### 빈도 표

In [34]:
freq_wpm = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'n': tdm_wpm.sum(axis=0).flat
})

### 결과 보기

In [35]:
freq_wpm.sort_values('n').tail(10)

,word,n
557,연구~,340
185,기술~,350
660,있다.,374
871,한다.,419
220,다.,433
883,했다.,440
850,하고,708
600,으로,734
546,에서,863
856,하는,912
